In [2]:
import pandas as pd
import numpy as np 

results = pd.read_csv("..\\Data\\results_norm.csv")
target = pd.read_csv("..\\Data\\fin_data.csv")
results = results.T
results = results.drop(["Unnamed: 0"])
target.index = target["Date"]

target = target.drop(["Date"], axis=1)
target = target.pct_change()
target = pd.DataFrame(np.log1p(target))

target = target.iloc[1:,:]
target = target.iloc[1000:, 0]
display(results)
display(target)

,0,1,2,3,4,5,6
0,-0.603833,-0.603833,-0.705414,-0.705414,-0.614431,-0.614431,0.147427
1,-0.075736,-0.075736,-0.010787,-0.010787,-0.093199,-0.093199,0.082646
2,-0.214238,-0.214238,-0.014975,-0.014975,-0.203381,-0.203381,-0.597889
3,1.003289,1.003289,1.072729,1.072729,1.058736,1.058736,0.203629
4,0.358964,0.358964,0.404655,0.404655,0.374246,0.374246,0.232869
...,...,...,...,...,...,...,...
4212,0.520512,0.675339,0.677557,0.409350,0.398789,0.409975,-0.619162
4213,0.015837,0.021820,-0.242217,0.294630,0.055508,0.078065,-0.425196
4214,-0.576375,-0.362277,-0.936974,-0.517001,-0.511433,-0.264340,0.305248
4215,-1.376787,-0.809702,-1.299833,-1.123831,-1.180431,-1.123033,0.082626


Date
2006-11-02   -0.000344
2006-11-03   -0.002226
2006-11-06    0.011283
2006-11-07    0.002215
2006-11-08    0.002081
                ...   
2022-12-26    0.000000
2022-12-27   -0.004058
2022-12-28   -0.012093
2022-12-29    0.017311
2022-12-30   -0.002544
Name: 0, Length: 4217, dtype: float64

In [8]:
from dieboldmariano import dm_test
from ARXT import hit_rate
from sklearn.metrics import mean_squared_error
from math import sqrt

def res(target, forecast):
    rmse = sqrt(mean_squared_error(target, forecast))
    hr = hit_rate(target, forecast)
    return(rmse, hr)

In [5]:
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame that includes a 'Date' column and a 'Performance_Metric' column
# Your list of changepoints
changepoints = [
    "2008-09-19",
    "2010-05-11",
    "2011-08-10",
    "2015-08-27",
    "2018-12-27",
    "2020-03-03",
    "2022-11-11",
]

def summary_splits(df, changepoints):
    # Convert changepoints to datetime
    changepoints = pd.to_datetime(changepoints)

    # Sort the DataFrame by date to ensure correct split calculations
    # df.sort_values(by='Date', inplace=True)

    # Convert the 'Date' column to datetime if it's not already
    df.index = pd.to_datetime(df.index)

    # Initialize the start date for the first split
    start_date = df.index.min()

    # Dictionary to hold the summary statistics for each split
    summary_statistics = {}

    # Iterate over the changepoints to create splits and calculate summary statistics
    for i, cp in enumerate(changepoints, start=1):
        # Create a mask for the current split
        mask = df[(df.index >= start_date) & (df.index < cp)]
        # Calculate summary statistics for the current split
        summary_statistics[f'Split {i}'] = mask.describe()
        # Update the start date for the next split
        start_date = cp

    # The last split goes from the last changepoint to the latest date
    mask = df[(df.index >= start_date)]
    summary_statistics[f'Split {i+1}'] = mask.describe()

    # Convert summary statistics dictionary to a DataFrame for better display and analysis
    summary_stats_df = pd.DataFrame(summary_statistics)
    return summary_stats_df
# Print or output the summary statistics DataFrame
df = target
# df_diff = df.diff()
df_norm=target

# summary_stats_diff = summary_splits(df_diff,changepoints)
summary_stats_norm = summary_splits(df_norm,changepoints)

display(summary_stats_diff, summary_stats_norm)


,Split 1,Split 2,Split 3,Split 4,Split 5,Split 6,Split 7,Split 8
count,490.000000,427.000000,326.000000,1056.000000,870.000000,308.000000,703.000000,36.000000
mean,0.000087,0.000001,0.000010,-0.000008,0.000012,-0.000011,0.000013,-0.001569
std,0.017986,0.033519,0.016804,0.013220,0.011865,0.012261,0.025466,0.017571
min,-0.066122,-0.114908,-0.068383,-0.091474,-0.049856,-0.044749,-0.216461,-0.044754
25%,-0.009173,-0.014058,-0.008540,-0.006865,-0.005455,-0.006756,-0.010077,-0.014973
50%,-0.000741,-0.001153,-0.000174,-0.000527,-0.000402,-0.000885,-0.000397,-0.002193
75%,0.009862,0.013951,0.007986,0.006914,0.004974,0.005844,0.010020,0.012910
max,0.090717,0.144948,0.115276,0.090407,0.059133,0.058828,0.188753,0.032072


,Split 1,Split 2,Split 3,Split 4,Split 5,Split 6,Split 7,Split 8
count,491.000000,427.000000,326.000000,1056.000000,870.000000,308.000000,703.000000,36.000000
mean,-0.000256,-0.000093,0.000034,0.000477,0.000276,0.000730,0.000351,-0.000833
std,0.011622,0.022432,0.011336,0.009201,0.008365,0.008815,0.016161,0.011267
min,-0.048288,-0.094695,-0.068958,-0.045619,-0.041843,-0.045168,-0.127652,-0.025237
25%,-0.005221,-0.008898,-0.004161,-0.003697,-0.002450,-0.002655,-0.006517,-0.008459
50%,0.000533,0.001128,0.000201,0.000307,0.000209,0.000882,0.000551,-0.001394
75%,0.005688,0.008915,0.005372,0.005064,0.004071,0.005761,0.008193,0.006239
max,0.042429,0.109572,0.046317,0.045251,0.048403,0.045011,0.089683,0.030479


Normalised Data

In [9]:
# rmse_sample = (sqrt(mean_squared_error(d_val_cumsum, valid_prediction_cumsum)))

res_norm_all = np.zeros((2,results.shape[1]))
dm_res_norm_all = np.zeros((2,results.shape[1]-1))

for i, forecast in enumerate(results):
        rmse, hr = res(target, results.iloc[:,i])
        res_norm_all[0,i] = rmse
        res_norm_all[1,i] = hr
        if i != 6:
            dm_stat, p_value = dm_test(target, results.iloc[:,i], results.iloc[:,6])

            # Store results
            dm_res_norm_all[0,i] = dm_stat
            dm_res_norm_all[1,i] = p_value
    
display(pd.DataFrame(res_norm_all).round(2))
display(pd.DataFrame(dm_res_norm_all).round(2))

,0,1,2,3,4,5,6
0,1.24,1.23,1.21,1.27,1.25,1.17,0.87
1,0.33,0.33,0.33,0.33,0.33,0.33,0.47


,0,1,2,3,4,5
0,9.87,8.95,10.48,9.98,8.85,8.77
1,0.00,0.00,0.00,0.00,0.00,0.00


In [10]:
changepoints = [
    "2008-09-19",
    "2010-05-11",
    "2011-08-10",
    "2015-08-27",
    "2018-12-27",
    "2020-03-03",
    "2022-11-11",
]

lengths = summary_stats_norm.loc["count",:]
lengths = pd.concat([pd.Series([0]), lengths])
lenghts = lengths.cumsum()
lenghts.index = range(len(lenghts))

In [12]:

res_norm = np.zeros(((len(lengths)-1)*2,results.shape[1]))
dm_res_norm = np.zeros(((len(lengths)-1)*2,results.shape[1]-1))

for i, forecast in enumerate(results):
    for j in range(len(lengths)-1):
        rmse, hr = res(target.iloc[int(lenghts[j]):int(lenghts[j+1])], results.iloc[int(lenghts[j]):int(lenghts[j+1]),i])
        res_norm[2*j,i] = rmse
        res_norm[2*j+1,i] = hr
        if i != 6:
            dm_stat, p_value = dm_test(target.iloc[int(lenghts[j]):int(lenghts[j+1])], results.iloc[int(lenghts[j]):int(lenghts[j+1]),i], results.iloc[int(lenghts[j]):int(lenghts[j+1]),6])

            # Store results
            dm_res_norm[j*2,i] = dm_stat
            dm_res_norm[j*2+1,i] = p_value
    
display(pd.DataFrame(res_norm).round(2))
display(pd.DataFrame(dm_res_norm).round(2))

,0,1,2,3,4,5,6
0,1.10,1.10,1.14,1.14,1.04,1.04,0.72
1,0.31,0.31,0.31,0.31,0.32,0.32,0.51
2,2.25,2.15,2.14,2.24,2.28,1.99,1.36
3,0.34,0.33,0.33,0.33,0.35,0.32,0.46
4,1.06,1.09,1.15,1.16,1.18,1.01,0.69
5,0.33,0.34,0.35,0.33,0.33,0.33,0.46
6,0.91,0.90,0.91,0.94,0.95,0.85,0.73
7,0.34,0.33,0.33,0.33,0.32,0.33,0.47
8,0.80,0.82,0.80,0.82,0.84,0.83,0.63
9,0.33,0.35,0.34,0.34,0.33,0.32,0.50


,0,1,2,3,4,5
0,6.59,6.59,6.74,6.74,5.85,5.85
1,0.00,0.00,0.00,0.00,0.00,0.00
2,5.94,5.59,5.48,6.18,4.71,4.87
3,0.00,0.00,0.00,0.00,0.00,0.00
4,3.61,4.80,4.20,4.45,4.31,3.56
5,0.00,0.00,0.00,0.00,0.00,0.00
6,4.22,4.35,4.69,5.24,5.75,3.10
7,0.00,0.00,0.00,0.00,0.00,0.00
8,5.13,5.80,4.96,5.76,5.84,5.64
9,0.00,0.00,0.00,0.00,0.00,0.00


In [13]:
res_norm = pd.DataFrame(res_norm, columns=results.columns)
dm_res_norm = pd.DataFrame(dm_res_norm)
res_norm_all = pd.DataFrame(res_norm_all, columns=results.columns)
dm_res_norm_all = pd.DataFrame(dm_res_norm_all)

res_norm.columns = ["ARXT_exog_retuned", "ARXT_exog_retrained", "ARXT_target_retuned", "ARXT_target_retrained", "ARX_retrained", "ARX", "AR"]
res_norm = res_norm.round(2)
res_norm.index = [f"{i//2}_{'rmse' if i % 2 == 0 else 'hr'}" for i in range(len(res_norm))]
res_norm.to_csv("split_results_norm.csv")

res_norm_all.columns = ["ARXT_exog_retuned", "ARXT_exog_retrained", "ARXT_target_retuned", "ARXT_target_retrained", "ARX_retrained", "ARX", "AR"]
res_norm_all = res_norm_all.round(2)
res_norm_all.index = [f"{i//2}_{'rmse' if i % 2 == 0 else 'hr'}" for i in range(len(res_norm_all))]
res_norm_all.to_csv("split_results_norm_all.csv")

dm_res_norm.columns = ["ARXT_exog_retuned", "ARXT_exog_retrained", "ARXT_target_retuned", "ARXT_target_retrained", "ARX_retrained", "ARX"]
dm_res_norm = dm_res_norm.round(2)
dm_res_norm.index = [f"{i//2}_{'test_stat' if i % 2 == 0 else 'p_val'}" for i in range(len(dm_res_norm))]
dm_res_norm.to_csv("split_results_norm_db.csv")

dm_res_norm_all.columns = ["ARXT_exog_retuned", "ARXT_exog_retrained", "ARXT_target_retuned", "ARXT_target_retrained", "ARX_retrained", "ARX"]
dm_res_norm_all = dm_res_norm_all.round(2)
dm_res_norm_all.index = [f"{i//2}_{'test_stat' if i % 2 == 0 else 'p_val'}" for i in range(len(dm_res_norm_all))]
dm_res_norm_all.to_csv("split_results_norm_db_all.csv")

Differenced

In [147]:

res_diff_all = np.zeros((2,results.shape[1]))
dm_res_diff_all = np.zeros((2,results.shape[1]-1))

for i, forecast in enumerate(results):
        rmse, hr = res(target, results.iloc[:,i])
        res_diff_all[0,i] = rmse
        res_diff_all[1,i] = hr
        if i != 6:
            dm_stat, p_value = dm_test(target, results.iloc[:,i], results.iloc[:,6])

            # Store results
            dm_res_diff_all[0,i] = dm_stat
            dm_res_diff_all[1,i] = p_value
    
    
display(pd.DataFrame(res_diff_all).round(2))
display(pd.DataFrame(dm_res_diff_all).round(2))

,0,1,2,3,4,5,6
0,43.51,43.43,41.15,41.15,42.17,42.25,34.26
1,0.36,0.36,0.33,0.33,0.33,0.33,0.47


,0,1,2,3,4,5
0,5.03,4.99,4.5,4.5,4.86,4.91
1,0.00,0.00,0.0,0.0,0.00,0.00


In [149]:
res_diff = np.zeros(((len(lengths)-1)*2,results.shape[1]))
dm_res_diff = np.zeros(((len(lengths)-1)*2,results.shape[1]-1))

for i, forecast in enumerate(results):
    for j in range(len(lengths)-1):
        rmse, hr = res(target.iloc[int(lenghts[j]):int(lenghts[j+1])], results.iloc[int(lenghts[j]):int(lenghts[j+1]),i])
        res_diff[2*j,i] = rmse
        res_diff[2*j+1,i] = hr
        if i != 6:
            dm_stat, p_value = dm_test(target.iloc[int(lenghts[j]):int(lenghts[j+1])], results.iloc[int(lenghts[j]):int(lenghts[j+1]),i], results.iloc[int(lenghts[j]):int(lenghts[j+1]),6])

            # Store results
            dm_res_diff[j*2,i] = dm_stat
            dm_res_diff[j*2+1,i] = p_value
    
display(pd.DataFrame(res_diff).round(2))
display(pd.DataFrame(dm_res_diff).round(2))

,0,1,2,3,4,5,6
0,29.21,29.21,27.39,27.39,26.32,26.32,21.84
1,0.33,0.33,0.31,0.31,0.31,0.31,0.52
2,23.79,23.79,22.08,22.08,21.77,21.77,18.17
3,0.36,0.36,0.33,0.33,0.32,0.32,0.46
4,29.66,29.66,27.33,27.33,28.89,28.89,24.18
5,0.36,0.36,0.33,0.33,0.33,0.33,0.50
6,42.06,42.06,38.50,38.50,40.68,40.68,35.27
7,0.35,0.35,0.36,0.36,0.34,0.34,0.42
8,130.48,130.48,124.61,124.61,128.15,128.15,91.30
9,0.27,0.27,0.24,0.24,0.23,0.23,0.48


,0,1,2,3,4,5
0,4.82,4.82,3.88,3.88,3.37,3.37
1,0.00,0.00,0.00,0.00,0.00,0.00
2,7.54,7.54,6.77,6.77,6.23,6.23
3,0.00,0.00,0.00,0.00,0.00,0.00
4,4.95,4.95,3.44,3.44,4.37,4.37
5,0.00,0.00,0.00,0.00,0.00,0.00
6,1.90,1.90,0.97,0.97,1.55,1.55
7,0.06,0.06,0.33,0.33,0.12,0.12
8,2.11,2.11,2.16,2.16,2.25,2.25
9,0.04,0.04,0.03,0.03,0.03,0.03


In [151]:
res_diff = pd.DataFrame(res_diff, columns=results.columns)
dm_res_diff = pd.DataFrame(dm_res_diff)
res_diff_all = pd.DataFrame(res_diff_all, columns=results.columns)
dm_res_diff_all = pd.DataFrame(dm_res_diff_all)

res_diff.columns = ["ARXT_exog_retuned", "ARXT_exog_retrained", "ARXT_target_retuned", "ARXT_target_retrained", "ARX_retrained", "ARX", "AR"]
res_diff = res_diff.round(2)
res_diff.index = [f"{i//2}_{'rmse' if i % 2 == 0 else 'hr'}" for i in range(len(res_diff))]
res_diff.to_csv("split_results_diff.csv")

res_diff_all.columns = ["ARXT_exog_retuned", "ARXT_exog_retrained", "ARXT_target_retuned", "ARXT_target_retrained", "ARX_retrained", "ARX", "AR"]
res_diff_all = res_diff_all.round(2)
res_diff_all.index = [f"{i//2}_{'rmse' if i % 2 == 0 else 'hr'}" for i in range(len(res_diff_all))]
res_diff_all.to_csv("split_results_diff_all.csv")

dm_res_diff.columns = ["ARXT_exog_retuned", "ARXT_exog_retrained", "ARXT_target_retuned", "ARXT_target_retrained", "ARX_retrained", "ARX"]
dm_res_diff = dm_res_diff.round(2)
dm_res_diff.index = [f"{i//2}_{'test_stat' if i % 2 == 0 else 'p_val'}" for i in range(len(dm_res_diff))]
dm_res_diff.to_csv("split_results_diff_db.csv")

dm_res_diff_all.columns = ["ARXT_exog_retuned", "ARXT_exog_retrained", "ARXT_target_retuned", "ARXT_target_retrained", "ARX_retrained", "ARX"]
dm_res_diff_all = dm_res_diff_all.round(2)
dm_res_diff_all.index = [f"{i//2}_{'test_stat' if i % 2 == 0 else 'p_val'}" for i in range(len(dm_res_diff_all))]
dm_res_diff_all.to_csv("split_results_diff_db_all.csv")